<a href="https://colab.research.google.com/github/felipecadm/carteira-de-ativos/blob/main/Aloca%C3%A7%C3%A3o_e_Precifica%C3%A7%C3%A3o.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px


In [3]:
acoes = pd.read_csv('acoes.csv')
acoes

,Date,EDP,ITSA,PETR,IBOV
0,2015-02-02,8.795126,6.545879,8.790000,46.270000
1,2015-02-03,8.479606,6.758638,9.970000,47.639999
2,2015-02-04,8.321846,7.006857,10.020000,47.770000
3,2015-02-05,8.183806,7.006857,9.830000,47.880001
4,2015-02-06,8.085206,7.091960,9.120000,47.299999
...,...,...,...,...,...
1886,2022-09-02,22.879999,9.300000,33.419998,107.089996
1887,2022-09-05,22.660000,9.410000,33.340000,108.449997
1888,2022-09-06,22.580000,9.250000,32.099998,105.910004
1889,2022-09-08,22.450001,9.270000,31.799999,106.180000


In [4]:
acoes.set_index('Date', inplace=True)
acoes

,EDP,ITSA,PETR,IBOV
Date,,,,
2015-02-02,8.795126,6.545879,8.790000,46.270000
2015-02-03,8.479606,6.758638,9.970000,47.639999
2015-02-04,8.321846,7.006857,10.020000,47.770000
2015-02-05,8.183806,7.006857,9.830000,47.880001
2015-02-06,8.085206,7.091960,9.120000,47.299999
...,...,...,...,...
2022-09-02,22.879999,9.300000,33.419998,107.089996
2022-09-05,22.660000,9.410000,33.340000,108.449997
2022-09-06,22.580000,9.250000,32.099998,105.910004


In [5]:
def alocacao_ativos(acoes, investimento, aleatoriedade = 0):
  dataset = acoes.copy()

  if aleatoriedade !=0:
    np.random.seed(aleatoriedade)
  
  pesos = np.random.random(len(dataset.columns))
  pesos = pesos / pesos.sum()
  print(pesos, pesos.sum())

  balanco_carteira = pd.DataFrame(data = {'Ações': dataset.columns, 'Pesos': pesos * 100})

  # Normalizar
  for i in dataset.columns:
    acoes[i] = (acoes[i] / acoes[i][0])

  # Aplicar pesos
  for i, acao in enumerate(dataset.columns):
    dataset[acao] = dataset[acao] * pesos[i] * investimento
  
  dataset['TOTAL'] = dataset.sum(axis=1)
  dataset['TX RETORNO'] = 0.0

  for i in range(1, len(dataset)):
    dataset['TX RETORNO'][i] = ((dataset['TOTAL'][i] / dataset['TOTAL'][i-1]) - 1) * 100

  


  return dataset, balanco_carteira

In [18]:
acoes_normalizado, balanco_carteira = alocacao_ativos(acoes, 5000, 6)

[0.427663   0.159012   0.39335311 0.01997189] 1.0


In [19]:
balanco_carteira

,Ações,Pesos
0,EDP,42.766300
1,ITSA,15.901200
2,PETR,39.335311
3,IBOV,1.997189


In [20]:
acoes_normalizado

,EDP,ITSA,PETR,IBOV,TOTAL,TX RETORNO
Date,,,,,,
2015-02-02,2138.315016,795.060012,1966.765526,99.859447,5000.000000,0.000000
2015-02-03,2061.604145,820.901643,2230.791001,102.816165,5216.112954,4.322259
2015-02-04,2023.248826,851.050232,2241.978561,103.096732,5219.374351,0.062525
2015-02-05,1989.687951,851.050232,2199.465876,103.334134,5143.538193,-1.452974
2015-02-06,1965.715731,861.386815,2040.603123,102.082380,4969.788049,-3.378028
...,...,...,...,...,...,...
2022-09-02,5562.699838,1129.574560,7477.736126,231.120547,14401.131071,-0.639699
2022-09-05,5509.212505,1142.935077,7459.836543,234.055686,14346.039811,-0.382548
2022-09-06,5489.762523,1123.501555,7182.385739,228.573898,14024.223716,-2.243240


In [21]:
retorno_diário = px.line(x = acoes_normalizado.index, y=acoes_normalizado['TX RETORNO'], title='Retorno Diário 2020 - Atualmente')
retorno_diário.show()

In [22]:
Valorizacao = px.line(title = 'Evolução do Patrimônio')
for ativo in acoes_normalizado.columns[0:4]:
  Valorizacao.add_scatter(x = acoes_normalizado.index, y = acoes_normalizado[ativo], name = ativo)
Valorizacao.show()